In [1]:
import spacy
from spacy.lang.sv import Swedish
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
import os 
import json
from bs4 import BeautifulSoup
import numpy as np
import string

stop_words = spacy.lang.sv.stop_words.STOP_WORDS
punctuations = string.punctuation

nlp = spacy.load('sv_core_news_lg')

def swedish_tokenizer(text):
    doc = nlp(text)
    return [token.lower_ for token in doc if token.text not in stop_words and token.text not in punctuations]

tfidf_vectorizer = TfidfVectorizer(tokenizer=swedish_tokenizer,min_df=5)

svd = TruncatedSVD(n_components=128)

log_reg = LogisticRegression(multi_class="multinomial", max_iter=1000)

<p>Extract the data in a very simple fashion.</p>

In [2]:
speeches = []
parties = []

possible_parties = ["S","M","SD","V","MP","C","KD","L"]

# The directory where data is located
data_dirs = ["speech_data/19_20/","speech_data/20_21/","speech_data/21_22/","speech_data/22_23/"]
year_indices = [0,0,0,0]
speeches_per_year = 1000
data = []
i = 0
for year, data_dir in enumerate(data_dirs):
    year_indices[year] = len(speeches)
    speech_count = 0
    for file in os.listdir(data_dir):
        if file.endswith(".json"):
            if speech_count >= speeches_per_year:
                break
            with open(data_dir+file,"r") as f:
                data = json.load(f)
                speech = data["anforande"]["anforandetext"]
                party = data["anforande"]["parti"]
                # If it is a party who has spoken,
                if party in possible_parties:
                    soup = BeautifulSoup(speech, 'html.parser')
                    speech_text = " ".join([p.get_text() for p in soup.find_all("p")])
                    if speech_text != "":
                        parties.append(party)
                        speeches.append(speech_text)
                        speech_count += 1

print(year_indices)
assert len(parties) == len(speeches)

[0, 1000, 2000, 3000]


In [3]:
speeches = np.array(speeches)
parties = np.array(parties)
print(speeches[:10])
print(parties[:10])

['Fru talman! Ann-Christine From Utterstedt gör sig nu skyldig till att blanda siffror och tider. Den siffran stämmer inte. Vi har inte den typen av överdödlighet i det här läget. Tvärtom ser vi nu att smittspridningen minskar och att antalet som vårdas i vanlig sjukvård och intensivvård minskar. Det gör att vi nu kan ta ytterligare steg framåt och börja med testning i stor skala, vilket regeringen nu har aviserat att vi kommer att se till att det finns resurser för. Vi kommer också att tillsätta en kommission som ska utvärdera pandemin och de åtgärder som vidtagits. Den kommer att ge resultat framöver.'
 'Fru talman! Tack, Lena Micko, för svaret! Jag måste dock börja med att ifrågasätta påståendet i ministerns svar att nyanlända etablerar sig i en snabbare takt på arbetsmarknaden. Det är faktiskt vilseledande att påstå detta. Enligt en rapport jag själv har begärt från riksdagens utredningstjänst är det efter två år i etableringsreformen bara 6 procent av de nyanlända som finns i ett 

In [4]:
# Turn parties to index
party_to_number = {}
number_to_party = {}
for i, p in enumerate(possible_parties):
    party_to_number[p] = i
    number_to_party[i] = p

print(party_to_number)
print(number_to_party)
print(parties[:20])
parties = [party_to_number.get(p) for p in parties]
print(parties[:20])

{'S': 0, 'M': 1, 'SD': 2, 'V': 3, 'MP': 4, 'C': 5, 'KD': 6, 'L': 7}
{0: 'S', 1: 'M', 2: 'SD', 3: 'V', 4: 'MP', 5: 'C', 6: 'KD', 7: 'L'}
['MP' 'M' 'C' 'S' 'S' 'V' 'V' 'S' 'KD' 'C' 'SD' 'SD' 'S' 'M' 'S' 'S' 'M'
 'SD' 'KD' 'M']
[4, 1, 5, 0, 0, 3, 3, 0, 6, 5, 2, 2, 0, 1, 0, 0, 1, 2, 6, 1]


In [3]:
#train_stop = year_indices[3]
#test_stop = len(speeches)

train_stop = speeches_per_year*3
test_stop = speeches_per_year*4

X_train = speeches[:train_stop]
y_train = parties[:train_stop]

X_test = speeches[train_stop:train_stop+test_stop]
y_test = parties[train_stop:train_stop+test_stop]

NotFittedError: Vocabulary not fitted or provided

In [5]:
print(X_train[:10])
print(X_train.shape)
print(X_test[:10])
print(X_test.shape)

['Fru talman! Ann-Christine From Utterstedt gör sig nu skyldig till att blanda siffror och tider. Den siffran stämmer inte. Vi har inte den typen av överdödlighet i det här läget. Tvärtom ser vi nu att smittspridningen minskar och att antalet som vårdas i vanlig sjukvård och intensivvård minskar. Det gör att vi nu kan ta ytterligare steg framåt och börja med testning i stor skala, vilket regeringen nu har aviserat att vi kommer att se till att det finns resurser för. Vi kommer också att tillsätta en kommission som ska utvärdera pandemin och de åtgärder som vidtagits. Den kommer att ge resultat framöver.'
 'Fru talman! Tack, Lena Micko, för svaret! Jag måste dock börja med att ifrågasätta påståendet i ministerns svar att nyanlända etablerar sig i en snabbare takt på arbetsmarknaden. Det är faktiskt vilseledande att påstå detta. Enligt en rapport jag själv har begärt från riksdagens utredningstjänst är det efter två år i etableringsreformen bara 6 procent av de nyanlända som finns i ett 

In [7]:
X_vectorized = tfidf_vectorizer.fit_transform(X_train[:100])
#X_vectorized = np.array(X_vectorized)


In [8]:
tfidf_vectorizer.get_feature_names_out()

array([' ', '  ', '000', '1', '10', '100', '18', '2', '20', '200', '2019',
       '2020', '5', '50', '60', 'absolut', 'agera', 'allra', 'alls',
       'allvarlig', 'anförande', 'annars', 'annorlunda', 'anser',
       'anställda', 'ansvar', 'ansvaret', 'antal', 'antalet', 'använda',
       'används', 'applåder', 'arbeta', 'arbetar', 'arbete', 'arbetet',
       'arbetsförmedlingen', 'avser', 'avslutad', 'barn', 'befolkning',
       'behov', 'behöver', 'behövs', 'beredd', 'betala', 'betydligt',
       'betänkandet', 'bidra', 'bifall', 'biståndet', 'bor', 'borde',
       'bred', 'brister', 'brott', 'brukar', 'budget', 'budgeten',
       'bygga', 'bygger', 'bästa', 'både', 'bör', 'börja', 'början',
       'börjar', 'centerpartiet', 'dagens', 'debatt', 'debatten', 'delar',
       'delta', 'denna', 'dessa', 'dessutom', 'dialog', 'diskussion',
       'diskutera', 'drar', 'därefter', 'däremot', 'därmed', 'effekt',
       'efterfrågan', 'egen', 'egentligen', 'egna', 'ekonomin',
       'ekonomisk

In [14]:
print(X_vectorized.shape)
print(tfidf_vectorizer.inverse_transform(X_vectorized[0]))
print(speeches[0])
print(swedish_tokenizer(speeches[0]))

(100, 476)
[array(['framöver', 'ge', 'åtgärder', 'resurser', 'se', 'regeringen',
       'börja', 'framåt', 'steg', 'ytterligare', 'ta', 'antalet',
       'minskar', 'ser', 'tvärtom', 'siffror', 'talman', 'fru'],
      dtype='<U19')]
Fru talman! Ann-Christine From Utterstedt gör sig nu skyldig till att blanda siffror och tider. Den siffran stämmer inte. Vi har inte den typen av överdödlighet i det här läget. Tvärtom ser vi nu att smittspridningen minskar och att antalet som vårdas i vanlig sjukvård och intensivvård minskar. Det gör att vi nu kan ta ytterligare steg framåt och börja med testning i stor skala, vilket regeringen nu har aviserat att vi kommer att se till att det finns resurser för. Vi kommer också att tillsätta en kommission som ska utvärdera pandemin och de åtgärder som vidtagits. Den kommer att ge resultat framöver.
['fru', 'talman', 'ann-christine', 'from', 'utterstedt', 'skyldig', 'blanda', 'siffror', 'tider', 'den', 'siffran', 'stämmer', 'vi', 'typen', 'överdödlighet',

In [ ]:
import torch
torch.save(X_vectorized,f="raw_tfidf.pt")

In [8]:
"""from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
X_norm = normalizer.fit_transform(X_vectorized)
print("normalization done")"""
svd.fit(X_vectorized)
X_train = svd.transform(X_vectorized)

In [ ]:
print(X_train[0])
print(X_train.shape)

[ 0.08588267  0.05020986  0.00343951  0.05347459  0.02333855  0.00047615
 -0.03864607 -0.00042119 -0.01756879  0.01065985  0.03719612 -0.00252954
  0.02429816 -0.03417375 -0.06224575 -0.02457703 -0.01085342 -0.00994339
 -0.01728675  0.04507677 -0.01920429  0.00341986 -0.07127325 -0.03484817
  0.0348405   0.0330292   0.00035511 -0.00271605 -0.0002963   0.02713186
 -0.03189182 -0.06350884 -0.02341523 -0.02418795 -0.02341454 -0.02387397
 -0.03879796  0.0380549  -0.01573021 -0.01141986  0.02189467  0.04540593
  0.07767848 -0.00466023 -0.02681386 -0.04831742  0.01726083  0.01205828
  0.0013939   0.02810338 -0.03135471  0.06637329 -0.01635103  0.00802815
 -0.01243777  0.00806794 -0.02101066  0.01346689 -0.02570184 -0.00953067
 -0.02435411  0.0181724  -0.0418033  -0.02384725  0.02947393  0.07102181
  0.0314088   0.03583948 -0.04074807 -0.00229019  0.08619431 -0.01740448
  0.05053666 -0.06001461  0.05766582 -0.08596859 -0.00734631  0.03451282
  0.02104285  0.03637774  0.01188429  0.00075407 -0

In [ ]:
X_test = svd.transform(normalizer.transform(tfidf_vectorizer.transform(X_test)))

In [ ]:
print(X_test.shape)
print(X_test[0])

(500, 256)
[ 2.89067412e-01  1.19244732e-02 -4.60951316e-02  1.29153800e-02
 -7.67164905e-02 -2.31782061e-02 -2.82930070e-02 -4.02281644e-02
 -4.12916453e-02  1.13390865e-02  2.76548730e-02  1.70310735e-02
  2.63621209e-02  6.35427690e-03  8.92531022e-03 -9.76586698e-03
 -1.19823728e-02  9.08233478e-03  8.31227757e-03 -3.91490977e-04
  5.25134946e-03  1.53734629e-02  4.50812936e-02 -1.22483047e-02
  1.65654859e-02 -2.57024541e-02 -3.41408164e-02  1.22184531e-02
  4.72705463e-03 -1.43194022e-02 -2.55318417e-03 -7.93004396e-03
  3.88387948e-02  9.59821032e-03  1.06865905e-02 -3.33933302e-03
 -2.13066665e-02  1.74323208e-02  6.78821968e-03 -1.23318301e-03
  1.50075421e-02  2.50892076e-02 -1.05774876e-02 -6.56855397e-03
  3.68716274e-03  6.06765259e-04 -7.30080145e-03 -1.22575623e-02
  2.56038741e-02 -1.21684650e-02 -1.20837689e-02  1.95588135e-02
  2.06869882e-02 -1.97692330e-02 -5.18129077e-03 -5.49568222e-03
 -1.88185820e-02  1.22754822e-02  1.81018034e-03 -9.52726047e-03
 -1.95809515e-

In [50]:
log_reg = LogisticRegression(multi_class="multinomial", max_iter=10000)

log_reg.fit(X_train,y_train)

LogisticRegression(max_iter=10000, multi_class='multinomial')

In [52]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = log_reg.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           C       1.00      0.03      0.06        96
          KD       0.00      0.00      0.00       101
           L       0.00      0.00      0.00        72
           M       0.58      0.56      0.57       269
          MP       0.71      0.04      0.07       132
           S       0.44      0.99      0.61       511
          SD       0.70      0.30      0.42       203
           V       0.80      0.07      0.13       116

    accuracy                           0.49      1500
   macro avg       0.53      0.25      0.23      1500
weighted avg       0.54      0.49      0.39      1500



/home/erki/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/erki/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/erki/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
